## API실습

In [57]:
# get() 메서드 -> 첫번째는 URL , 두번쨰는 keyword
import requests 

response = requests.get("https://api.github.com/repositories",headers={'Accept': "application/vnd.github+json"})

print(response.status_code) # 통신이 잘 되었는지 확인하는 코드 

200


In [65]:
print('인코딩 : {}'.format(response.encoding))
print('콘텐처 타입: {}'.format(response.headers['content-Type']))
print('서버 : {}'.format(response.headers['server']))

인코딩 : utf-8
콘텐처 타입: application/json; charset=utf-8
서버 : GitHub.com


In [68]:
import json
print(json.dumps(response.json()[0],indent=2)[:200]) # 딕셔너리 형태로 만들어서 200자만 봄

{
  "id": 1,
  "node_id": "MDEwOlJlcG9zaXRvcnkx",
  "name": "grit",
  "full_name": "mojombo/grit",
  "private": false,
  "owner": {
    "login": "mojombo",
    "id": 1,
    "node_id": "MDQ6VXNlcjE=",



In [76]:
response.headers

{'Server': 'GitHub.com', 'Date': 'Mon, 27 Feb 2023 05:42:20 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept, Accept-Encoding, Accept, X-Requested-With', 'ETag': 'W/"3c54a93639294e719c717ee9fb29aa31617be2a77aed1b49a769e1b1c9dd5ef5"', 'X-GitHub-Media-Type': 'github.v3; format=json', 'Link': '<https://api.github.com/repositories?since=369>; rel="next", <https://api.github.com/repositories{?since}>; rel="first"', 'x-github-api-version-selected': '2022-11-28', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, Sunset', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Frame-Options': 'deny', 'X-

In [81]:
response = requests.get('https://api.github.com/search/repositories')
print(response.status_code) #서버가 처리를 못하는 요청인경우

422


### 새로운 API 가져오기

- parames 적는 공식    params={'q: '찾는 규격 이름'+language:python'}

In [91]:
response = requests.get('https://api.github.com/search/repositories',
                       params ={'q' :'data_science+language:python'},
                        headers={'Accept':"application/vnd.github+json"})
print(response.status_code)

200


In [92]:
response.headers

{'Server': 'GitHub.com', 'Date': 'Mon, 27 Feb 2023 07:12:31 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Cache-Control': 'no-cache', 'Vary': 'Accept, Accept-Encoding, Accept, X-Requested-With', 'X-GitHub-Media-Type': 'github.v3; format=json', 'Link': '<https://api.github.com/search/repositories?q=data_science%2Blanguage%3Apython&page=2>; rel="next", <https://api.github.com/search/repositories?q=data_science%2Blanguage%3Apython&page=34>; rel="last"', 'x-github-api-version-selected': '2022-11-28', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, Sunset', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 

In [93]:
response.headers['content-Type'] #어떻게 생긴 애인지 확인 함 -> json으로 되어있음

'application/json; charset=utf-8'

In [ ]:
import json #문자열을 파이썬의 자료형 형태로 변환해줌
response.json() #response 를 json() 매서드를 활용해 dict 타입으로 변경

In [94]:
response.json().keys()  # response 를 json으로 구조화 한것중 keys만 출력

dict_keys(['total_count', 'incomplete_results', 'items'])

In [98]:
len(response.json()['items'])

30

In [103]:
response.json()['items'][0]['name'] #items 중 -> 0번째 인덱스 중 -> name은 무엇인지

26382146

In [104]:
response.json()['items'][0]['id'] #items 중 -> 0번쨰 인덱스 중 -> id는 무엇인지

26382146

## 새로운 API 가져오기(2) - Issues 

- /repos/{owner}/{repo}/issues  (작성 형식)

In [148]:
response = requests.get('https://api.github.com/repos/pytorch/pytorch/issues',
                        headers={'Accept': "application/vnd.github+json"}) 
# get방식으로 가져오고 owner , repo 부분에 내가 가져오고 싶은 issue 정보 넣어 url 만들고 accept 넣으면 된다
print(response.status_code) # 실행 잘 되었는지
print(len(response.json())) # 총 30개만 가지고왔음

403
2


In [149]:
response.links #응답 페이지 수에대한 세부 정보를 제공

# next 는 30개 결과를 가져다주는 URL 제공
# last 는 총 검색 결과가 몇 개 있는지를 나타내는 마지막 페이지 URL 제공

{}

In [150]:
# 다음 링크가 있다면 계속 API 호출하는 함수 만들기
def get_all_pages(url, params=None, headers=None):
    output_json=[] #결과를 담음
    response = requests.get(url, params=params, headers=headers) # 요청을 함
    if response.status_code == 200: # 성공이라면
        output_json = response.json() # response.json() 결과를 담아라
        if 'next' in response.links: 
            next_url = response.links['next']['url']
            if next_url is not None:
                output_json += get_all_pages(next_url, params, headers)
    return output_json

In [151]:
import pandas as pd

out = get_all_pages('https://api.github.com/repos/pytorch/pytorch/issues',params={'since': '2022-01-01T10:00:01Z',
        'sorted': 'created','direction': 'desc'},headers={'Accept': "application/vnd.github+json"})

# params 은 2022-01-01 이후에 데이터만 가지고오며 셍성된 것들중 내림차순으로 정렬을 한다

df = pd.DataFrame(out) #호출값을 데이터 프레임으로 만듬 
df['id','created_at','body']].sample(1) # id,created_at,body 컬럼을 가지는거 하나만 추출해봄 

""


In [157]:
print('x-Ratelimit_limit',response.headers['X-Ratelimit-Limit'])
print('x-Ratelimit_Remaining',response.headers['X-Ratelimit-Remaining'])
# 시간당 몇개 주는지 

x-Ratelimit_limit 60
x-Ratelimit_Remaining 0


In [ ]:
# 호출 속도 조절 코드 (속도 조절을 하지 않으면 어떤 api는 너무 빨라서 비정상적 접근이라 출력할수도있음)
def handle_rate_limits(response):
    now = datetime.now()
    reset_time = dt.fromtimestamp(int(response.headers['X-RateLimit-Reset']))
    remaining_requests = response.headers['X-Ratelimit-Remaining']
    remaining_time =(reset_time-now).total_seconds()
    intervals = remaining_time / (1.0 +int(remaining_requests))
    
    print('Sleeping for', intervals)
    time.sleep(intervals)
    return True